In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

### Implémentation sous PyNN

Nous avons vu qu'une implémentation d'un même modèle peut radicalement changer de forme selon le simulateur utilisé. Il peut être utile, afin de ne pas passer trop de temps sur l'apprentissage des différents simulateurs, de trouver un moyen d'harmoniser l'écriture des modèles. PyNN (http://neuralensemble.org/PyNN) n'est pas un simulateur mais une API, une interface de description (i.e. programmation), qui permet de réaliser des simulations pouvant s'exécuter sur différents simulateurs, comme Nest ou Brian, sans changer le code d'implémentation. Elle permet également de générer divers graphiques aisément <cite data-cite="davis">(Davison et al., 2008)</cite>.

C'est avec cette interface que nous allons construire des réseaux neuronaux et programmer différents outils pour leur étude. Nous implémentons tout d'abord un réseau très simple afin de comparer l'efficacité de Nest et de Brian. Ces simulations permettent de vérifier que les implémentations de NEST et Brian fournissent des résultats quasi-identiques au niveau de l'implémentation des équations différentielles des modèles neuronaux. Il s'avère aussi que la simulation avec Nest se fait en un temps plus court que celle effectuée avec Brian. C'est pourquoi désormais nous utiliserons Nest pour la simulation de réseaux de neurones.

In [2]:
import pyNN.nest as sim
import numpy

from pyNN.utility import get_simulator, init_logging, normalized_filename
from pyNN.parameters import Sequence
from pyNN.random import RandomDistribution as rnd
from pyNN.utility.plotting import Figure, Panel


# === Define parameters ========================================================

n = 10      # Number of cells
w = 0.2  # synaptic weight (µS)
coba_params = {
    'tau_m'      : 20.0,   # (ms)
    'tau_syn_E'  : 2.0,    # (ms)
    'tau_syn_I'  : 4.0,    # (ms)
    'e_rev_E'    : 0.0,    # (mV)
    'e_rev_I'    : -70.0,  # (mV)
    'tau_refrac' : 2.0,    # (ms)
    'v_rest'     : -60.0,  # (mV)
    'v_reset'    : -70.0,  # (mV)
    'v_thresh'   : -50.0,  # (mV)
    'cm'         : 0.5}    # (nF)

cuba_params = {
    'tau_m'      : 20.0,   # (ms)
    'tau_syn_E'  : 2.0,    # (ms)
    'tau_syn_I'  : 4.0,    # (ms)
    'tau_refrac' : 2.0,    # (ms)
    'v_rest'     : -60.0,  # (mV)
    'v_reset'    : -70.0,  # (mV)
    'v_thresh'   : -50.0,  # (mV)
    'cm'         : 0.5,    # (nF)
    'i_offset'   : 0.0}
dt         = 0.1           # (ms)
syn_delay  = 1.0           # (ms)
input_rate = 10.0         # (Hz)
simtime    = 1000.0        # (ms)

# === Build the network ========================================================

sim.setup()

coba = sim.Population(n, sim.IF_cond_alpha(**coba_params),
                       initial_values={'v': rnd('uniform', (-60.0, -50.0))},
                       label="coba")

cuba = sim.Population(n, sim.IF_curr_alpha(**cuba_params),
                       initial_values={'v': rnd('uniform', (-60.0, -50.0))},
                       label="cuba")


number = int(2*simtime*input_rate/1000.0)
numpy.random.seed(26278342)


def generate_spike_times(i):
    gen = lambda: Sequence(numpy.add.accumulate(numpy.random.exponential(1000.0/input_rate, size=number)))
    if hasattr(i, "__len__"):
        return [gen() for j in i]
    else:
        return gen()
#assert generate_spike_times(0).max() > simtime

spike_source = sim.Population(n, sim.SpikeSourceArray(spike_times=generate_spike_times))


input_conn1 = sim.Projection(spike_source, coba, sim.FixedProbabilityConnector(.1), sim.StaticSynapse(weight=w/4.,delay=syn_delay))
input_conn2 = sim.Projection(spike_source, cuba, sim.FixedProbabilityConnector(.1), sim.StaticSynapse(weight=w*10,delay=syn_delay))

#-------- Recording -------------
spike_source.record('spikes')

coba.record('spikes')
coba[0:n].record(('v', 'gsyn_exc'))

cuba.record('spikes')
cuba[0:n].record(('v'))

# === Run simulation ===========================================================

sim.run(simtime)

print("COBA Mean firing rate: ", coba.mean_spike_count()*1000.0/simtime, "Hz")
print("CUBA Mean firing rate: ", cuba.mean_spike_count()*1000.0/simtime, "Hz")

# === Clean up and quit ========================================================

sim.end()

#### COBA

In [3]:
coba_data = coba.get_data().segments[0]
coba_vm = coba_data.filter(name="v")[0]
coba_gsyn = coba_data.filter(name="gsyn_exc")[0]
Figure(
    Panel(coba_vm, ylabel="Membrane potential (mV)"),
    Panel(coba_gsyn, ylabel="Synaptic conductance (uS)"),
    #Panel(coda_data.spiketrains, xlabel="Time (ms)", xticks=True, ms=1)
)


#### CUBA

In [4]:
cuba_data = cuba.get_data().segments[0]
cuba_vm = cuba_data.filter(name="v")[0]

_ = Figure(
    Panel(cuba_vm, ylabel="Membrane potential (mV)"),
#Panel(cuba_data.spiketrains, xlabel="Time (ms)", xticks=True, ms=1)
)

#### merge

In [5]:
Figure(
    Panel(coba_vm, ylabel="COBA: Membrane potential (mV)"),
    Panel(coba_gsyn, ylabel="COBA: Synaptic conductance (uS)"),
    Panel(cuba_vm, ylabel="CUBA: Membrane potential (mV)"),
    Panel(cuba_data.spiketrains, ylabel="CUBA: Spikes (mV)", xlabel="Time (ms)", xticks=True, ms=1)
).save('figs/pynn.png')